# L5: Content Creation at Scale

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'agents.yaml',
    'tasks': 'tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: I need to gather information about the latest news and updates related to Inflation in the US and the impact on the stock market in 2024, as this will be crucial in creating a summary report that highlights the most impactful financial news and updates.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest news on US inflation 20



# Agent: Chief Data Strategist
## Thought: Thought: To analyze market data and trends related to Inflation in the US and the impact on the stock market in 2024, I need to gather information from reputable sources.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Inflation in the US 2024 impact on stock market\"}"
## Tool Output: 

Search results: Title: How Inflation Affects The Stock Market | Bankrate
Link: https://www.bankrate.com/investing/how-inflation-affects-the-stock-market/
Snippet: But higher inflation rates, typically above 3 percent, could increase volatility across the economy and stock market. Inflation, especially at ...
---
Title: Schwab Market Perspective: After the Landing
Link: https://www.schwab.com/learn/story/stock-market-outlook
Snippet: In "real," or inflation-adjusted, terms, both expanded to 3.3% year-over-year in the second quarter of 2024. That is near the upper end of the range in the pre ...
---
Title: US Economic Forecast Q3 2024 

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]




# Agent: Chief Data Strategist
## Thought: Thought: After analyzing the search results and getting insights from various reputable sources, I have compiled a comprehensive analysis report related to Inflation in the US and the impact on the stock market in 2024.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Inflation in the US 2024 impact on stock market analysis\", \"website\": \"https://www.investopedia.com/\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the story is still positive overall. Consumer spending is forecasted to rise 2.



# Agent: Creative Content Director
## Thought: I need to gather the latest insights and data on inflation in the US and its projected impact on the stock market in 2024 to create comprehensive and engaging content.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Inflation in the US impact on stock market 2024 forecast analysis\"}"
## Tool Output: 

Search results: Title: US Economic Forecast Q3 2024 | Deloitte Insights
Link: https://www2.deloitte.com/us/en/insights/economy/us-economic-forecast/united-states-outlook-analysis.html
Snippet: The Q3 2024 forecast indicates how robust consumer spending, high business investment, and lower interest rates have kept optimism about the US economy intact.
---
Title: 2024 Economic Outlook: Insights & Trends | J.P. Morgan
Link: https://www.jpmorgan.com/insights/outlook/economic-outlook/economic-trends
Snippet: We forecast core PCE prices—the Fed's preferred inflation metric—to rise 2.4% in 2024, down from 3.4% in 2023.
--

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00, 49.41it/s]




# Agent: Creative Content Director
## Thought: Thought: I need to gather more specific insights and trends related to inflation and its impact on the stock market from the selected sources to create engaging content.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"inflation impact on stock market 2024 forecast analysis\", \"website\": \"https://www.deloitte.com\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the story is still positive overall. Consumer spending is forecasted to rise 2.3% this year, up from the 2.2% increase in 2023. Bus

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
🔍 **Navigating Inflation in 2024**  Inflation is
on the rise! Understanding its impact on the stock
market is crucial for investors. Explore how
Federal Reserve policies will shape market trends
and uncover investment strategies in our latest
article! [Read more](#) 📈💡
--------------------------------------------------
LinkedIn
📊 **Impact of Inflation on the US Stock Market in
2024** As inflation rises, savvy investors need to
understand the potential fluctuations in the stock
market. Our latest blog delves into economic
trends, the Federal Reserve's monetary policies,
and actionable investment strategies for
navigating this landscape. [Read the full
article](#)
--------------------------------------------------
Instagram
✨ **Investing in an Inflated World**  Is inflation
affecting your investments? Our new article
highlights key economic trends and how to position
your portfolio for success in 2024. Link in bio
for insights on surviving and thriving amidst
rising prices! #Inve

## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Navigating Inflation: The Impact on the US Stock Market in 2024

As the US grapples with rising inflation figures, understanding the implications for the stock market in 2024 becomes critical for investors. Inflation, characterized by the increased prices of commodities and services, affects purchasing power and consumer behavior, leading to fluctuations in market dynamics. The Federal Reserve's monetary policies play a significant role in how inflation shapes market trends. In this article, we will dissect the economic indicators, Federal Reserve responses, and potential investment strategies in light of the current inflationary environment.

## Economic Trends Contributing to Inflation

Several economic trends are driving inflation rates higher in the US. Supply chain disruptions, increased demand post-pandemic, and labor shortages have all contributed to rising costs. These complications have put pressure on businesses to adjust prices, which reflects on inflation indices. According to the Consumer Price Index (CPI), marked increases in essentials like food, energy, and housing services have been observed. Understanding these trends is essential for predicting the stock market's response and enabling informed investment decisions.

Investors must closely monitor the Federal Reserve's actions regarding interest rates as they combat inflation. While an increase in rates may initially deter investment and slow economic growth, it could stabilize prices in the long run. Consequently, investors should be prepared for volatility in sectors that are sensitive to interest rate changes, such as real estate and technology. This volatility can create both risk and opportunity, making it vital to execute carefully considered strategies.

## Federal Reserve Policies and Market Reactions

The Federal Reserve’s approach to controlling inflation will significantly impact stock market performance in 2024. Following a period of maintaining low-interest rates, the Fed is now faced with the challenge of striking a balance between curbing inflation and fostering economic growth. Analysts predict that gradual rate increases will occur throughout the year, which could temper consumer spending but also stabilize growth in the long term.

Investors should anticipate that Fed decisions will lead to initial market reactions characterized by uncertainty. Sectors such as utilities and consumer staples are traditionally viewed as more stable during inflationary periods, as they tend to maintain consistent demand. However, opportunities may also arise in sectors poised for growth, such as renewable energy or technology, which could benefit from innovation-driven demand despite inflation pressures.

## Investment Strategies for 2024

Given the outlined trends and Fed policies, investors should consider adopting a diversified portfolio strategy in 2024. This approach will help mitigate risks associated with inflation and fluctuating interest rates. Investing in inflation-protected securities, such as Treasury Inflation-Protected Securities (TIPS) or commodities, could provide a buffer against rising prices. Additionally, equities in defensive sectors—like healthcare and consumer necessities—may offer reliable returns despite market fluctuations.

In light of potential volatility, investors should also prioritize research on companies with strong pricing power. Businesses that can pass on costs to consumers without suffering significant losses are likely to perform better in an inflationary environment. Combining Long-Term investments in these companies with a short-term trading strategy can allow investors to capture growth opportunities while managing risk.

### Conclusion

Navigating the complexities of inflation and its impact on the US stock market in 2024 requires careful attention and action. By understanding economic trends, monitoring Federal Reserve policies, and implementing knowledgeable investment strategies, investors can prepare for the challenges and opportunities that lie ahead. Continuous education and a proactive investment approach will be crucial for achieving financial success in this evolving landscape.